In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('Db-IMDB.db')

# 1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [2]:
#https://github.com/stegemang/sqlimdb
#https://github.com/theja-vanka/imdb-sql

list_of_directors = pd.read_sql_query("""
WITH
    Comedy_Movies AS
    (
        SELECT
             -- COUNT(*)  107
            MG.MID
        FROM
                    GENRE G
            JOIN    M_GENRE MG
            ON      G.GID = MG.GID
        WHERE
            TRIM(G.Name) LIKE '%Comedy%'
    ),
    Comedy_Movies_In_Leap_Yr AS
    (
        SELECT
            M.MID,
            M.title,
            CAST(SUBSTR(M.year,-4) AS UNSIGNED) year
        FROM
                    Comedy_Movies CM
            JOIN    Movie M
            ON      CM.MID = M.MID
        WHERE
            (CAST(SUBSTR(M.year,-4) AS UNSIGNED) %4 = 0 )
    )
   SELECT
        DISTINCT
        TRIM(P.Name) Director_Name,
        CM.title Movie_Name,
        CM.year
    FROM
                Comedy_Movies_In_Leap_Yr CM
        JOIN    M_Director MD
        ON      CM.MID = MD.MID
        JOIN    Person P
        ON      TRIM(MD.PID) = TRIM(P.PID)
    ORDER BY 
    year
""", conn)

list_of_directors

,Director_Name,Movie_Name,year
0,Satyen Bose,Jagriti,1956
1,Chetan Anand,Funtoosh,1956
2,Amit Mitra,Jagte Raho,1956
3,Mohan Segal,New Delhi,1956
4,R.K. Nayyar,Love in Simla,1960
5,Bimal Roy,Parakh,1960
6,S.U. Sunny,Kohinoor,1960
7,Ravindra Dave,Dulha Dulhan,1964
8,Kidar Nath Sharma,Chitralekha,1964
9,Shakti Samanta,Kashmir Ki Kali,1964


# 2. List the names of all the actors who played in the movie 'Anand' (1971)

In [5]:
df = pd.read_sql_query('''SELECT name from Person where pid IN 
                        (SELECT pid FROM m_cast where mid IN
                        (SELECT mid from movie where title = 'Anand'));''', conn)

In [6]:
df

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


# 3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [5]:
result = pd.read_sql_query("""
WITH
    ACTORS_BEFORE_1970 AS
    (
        SELECT
            DISTINCT
            TRIM(MC.PID) PID
        FROM
                    Movie M
            JOIN    M_Cast MC
            ON      M.MID = MC.MID
        WHERE
            CAST(SUBSTR(M.year,-4) AS UNSIGNED) < 1970
    ),
    ACTORS_AFTER_1990 AS
    (
        SELECT
            DISTINCT
            TRIM(MC.PID) PID
        FROM
                    Movie M
            JOIN    M_Cast MC
            ON      M.MID = MC.MID
        WHERE
            CAST(SUBSTR(M.year,-4) AS UNSIGNED) > 1990
    )
    SELECT
        DISTINCT
        TRIM(P.Name) Actor_Name
    FROM
                ACTORS_BEFORE_1970 A_1970
        JOIN    ACTORS_AFTER_1990 A_1990
        ON      A_1970.PID = A_1990.PID
        JOIN    Person P
        ON      A_1970.PID = TRIM(P.PID)
""", conn)
result

,Actor_Name
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni
5,Rakesh Sharma
6,Sanjay Dutt
7,Ric Young
8,Yusuf
9,Suhasini Mulay


# 4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [7]:
result=pd.read_sql_query("""SELECT
    DISTINCT
    TRIM(P.NAME) DIRECTOR_NAME,
    NM.NUM_OF_MOVIES_DIRECTED
FROM
            (SELECT 
                PID,
                COUNT(MID) NUM_OF_MOVIES_DIRECTED
            FROM
                M_Director
            GROUP BY
                PID
            HAVING
                NUM_OF_MOVIES_DIRECTED >= 10
            ) NM
    JOIN    PERSON P
    ON      TRIM(NM.PID) = TRIM(P.PID)
ORDER BY 
    NM.NUM_OF_MOVIES_DIRECTED DESC
""", conn)

In [8]:
result

,DIRECTOR_NAME,NUM_OF_MOVIES_DIRECTED
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Priyadarshan,30
3,Ram Gopal Varma,30
4,Vikram Bhatt,29
5,Hrishikesh Mukherjee,27
6,Yash Chopra,21
7,Basu Chatterjee,19
8,Shakti Samanta,19
9,Subhash Ghai,18


# 5. a. For each year count the number of movies in that year that had only feamle actors

In [10]:
fr = pd.read_sql_query('''select z.year, count(*)
from Movie z
where not exists (select *
                  from Person x, M_Cast xy
                  where x.PID = xy.PID and xy.MID = z.MID and x.Gender!='Female')
group by z.year''', conn)
fr

,year,count(*)
0,1939,1
1,1999,1
2,2000,1
3,2009,1
4,2012,1
5,2018,2


# 5. b. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [11]:
perc_query = pd.read_sql_query('''select a.year, a.c*100.00/b.c as percentage, b.c as total_overall
from (select z.year, count(*) as c
      from Movie z
      where not exists (select *
                        from Person x, M_Cast xy
                        where x.PID = xy.PID and xy.MID = z.MID and x.Gender!='Female')
      group by z.year) a,
     (select z.year, count(*) as c from Movie z group by z.year) b
where a.year=b.year
order by a.year''', conn)
perc_query

,year,percentage,total_overall
0,1939,50.000000,2
1,1999,1.515152,66
2,2000,1.562500,64
3,2009,0.909091,110
4,2012,0.900901,111
5,2018,1.923077,104


# 6. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [12]:
l_cast_query = pd.read_sql_query('''select x.title, count(distinct xy.PID) as c
from Movie x, M_Cast xy
where x.MID = xy.MID
group by x.MID, x.title
having not exists (select uv.MID
                   from M_Cast uv
                   group by uv.MID
                   having count(distinct uv.PID) > count(distinct xy.PID))''', conn)
l_cast_query

,title,c
0,Ocean's Eight,238


# 7. A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [13]:
decade_query = pd.read_sql_query('''select y.year, count(*)
from (select distinct x.year from Movie x) y,
     Movie z
where y.year <= z.year and z.year < y.year+10
group by y.year
having not exists (select y1.year
                   from (select distinct x1.year from Movie x1) y1, Movie z1
                   where y1.year <= z1.year and z1.year < y1.year+10
                   group by y1.year
                   having count(z1.MID) > count(z.MID))''', conn)
decade_query

,year,count(*)
0,2008,1205


# 8. Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

In [14]:
unemployed_query = pd.read_sql_query('''select Name from Person where PID 
    not in(select distinct(PID)from M_Cast as C1 natural join Movie as 
    M1 where exists(select MID from M_Cast as C2 natural join Movie as M2 where 
    C1.PID = C2.PID and (M2.year - 3) > M1.year and 
    not exists(select MID from M_Cast as C3 natural join 
    Movie as M3 where C1.PID = C3.PID and M1.year < M3.year and M3.year < M2.year)))''', conn)
unemployed_query

,Name
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch
3,Naomie Harris
4,Andy Serkis
5,Peter Mullan
6,Jack Reynor
7,Eddie Marsan
8,Tom Hollander
9,Matthew Rhys


# 9. Find all the actors that made more movies with Yash Chopra than any other director.

In [3]:
result = pd.read_sql_query("""
WITH
    YASH_CHOPRAS_PID AS
    (
        SELECT
            TRIM(P.PID) AS PID
        FROM
            Person P
        WHERE
            Trim(P.Name) LIKE '%Yash Chopra%'
    ),
    NUM_OF_MOV_BY_ACTOR_DIRECTOR AS
    (
        SELECT
            TRIM(MC.PID) ACTOR_PID,
            TRIM(MD.PID) DIRECTOR_PID,
            COUNT(DISTINCT TRIM(MD.MID)) AS NUM_OF_MOV
        FROM
            M_Cast MC,
            M_Director MD
        WHERE
            TRIM(MC.MID)= TRIM(MD.MID)
        GROUP BY
            ACTOR_PID,
            DIRECTOR_PID
    ),
    NUM_OF_MOVIES_BY_YC AS
    (
        SELECT
            NM.ACTOR_PID,
            NM.DIRECTOR_PID,
            NM.NUM_OF_MOV NUM_OF_MOV_BY_YC
        FROM
            NUM_OF_MOV_BY_ACTOR_DIRECTOR NM,
            YASH_CHOPRAS_PID YCP
            WHERE
            NM.DIRECTOR_PID = YCP.PID
    ),
    MAX_MOV_BY_OTHER_DIRECTORS AS
    (
        SELECT
            ACTOR_PID,
            MAX(NUM_OF_MOV) MAX_NUM_OF_MOV
        FROM
            NUM_OF_MOV_BY_ACTOR_DIRECTOR NM,
            YASH_CHOPRAS_PID YCP
        WHERE
            NM.DIRECTOR_PID <> YCP.PID 
        GROUP BY
            ACTOR_PID
    ),
    ACTORS_MOV_COMPARISION AS
    (
    SELECT
        NMY.ACTOR_PID,
        CASE WHEN NMY.NUM_OF_MOV_BY_YC > IFNULL(NMO.MAX_NUM_OF_MOV,0) THEN 'Y' ELSE 'N' END MORE_MOV_BY_YC
    FROM
        NUM_OF_MOVIES_BY_YC NMY
         LEFT OUTER JOIN
        MAX_MOV_BY_OTHER_DIRECTORS NMO
        ON
        NMY.ACTOR_PID = NMO.ACTOR_PID 
    )
    SELECT
        DISTINCT
        TRIM(P.Name) ACTOR_NAME
    FROM
        Person P
    WHERE
        TRIM(P.PID) IN (
            SELECT
                DISTINCT
                ACTOR_PID
            FROM
                ACTORS_MOV_COMPARISION
            WHERE
                MORE_MOV_BY_YC = 'Y'
        )
""", conn)

In [4]:
result

,ACTOR_NAME
0,Waheeda Rehman
1,Achala Sachdev
2,Yash Chopra
3,Vinod Negi
4,Chandni Jas Keerat
5,Shivaya Singh
6,Huzefa Gadiwala
7,Manish Arora
8,Pankaj Raina
9,Neetu Singh


# 10. The Shahrukh number of an actor is the length of the shortest path between the actor andShahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; allactors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [12]:
result = pd.read_sql_query("""
WITH 
    SHAHRUKH_0 AS
    (
        SELECT
            TRIM(P.PID) PID
        FROM
            Person P
        WHERE
            Trim(P.Name) like '%Shahrukh%'
    ),
    SHAHRUKH_1_MOVIES AS
    (
        SELECT
            DISTINCT
            TRIM(MC.MID) MID,
            S0.PID
        FROM
            M_Cast MC,
            SHAHRUKH_0 S0
        WHERE
            TRIM(MC.PID) = S0.PID
    ),
    SHAHRUKH_1_ACTORS AS
    (
        SELECT
            DISTINCT
            TRIM(MC.PID) PID
        FROM
            M_Cast MC,
            SHAHRUKH_1_MOVIES S1M
        WHERE
            TRIM(MC.MID) = S1M.MID AND
            TRIM(MC.PID) <> S1M.PID
    ),
    SHAHRUKH_2_MOVIES AS
    (
        SELECT
            DISTINCT
            TRIM(MC.MID) MID,
            S1A.PID
        FROM
            M_Cast MC,
            SHAHRUKH_1_ACTORS S1A
        WHERE
            TRIM(MC.PID) = S1A.PID
    )
    SELECT
        DISTINCT
        TRIM(MC.PID) PID,
        TRIM(P.Name) ACTOR_NAME
    FROM
        Person P,
        M_Cast MC,
        SHAHRUKH_2_MOVIES S2M
    WHERE
            TRIM(MC.PID) = TRIM(P.PID) AND
            TRIM(MC.MID) = S2M.MID AND
            TRIM(MC.PID) <> S2M.PID;
    
""", conn)

In [14]:
result

,PID,ACTOR_NAME
0,nm2951768,Freida Pinto
1,nm6467532,Caroline Christl Long
2,nm6071249,Rajeev Pahuja
3,nm3491108,Michelle Santiago
4,nm7509518,Jandre le Roux
5,nm5951787,Raj Awasti
6,nm5525290,Michael Chapman
7,nm8232648,James Heron
8,nm7247557,Alex Jaep
9,nm6631007,Marian Lorencik
